In [13]:
import pandas as pd
import geopandas as gpd
from siuba import *
import shared_utils

from calitp_data_analysis import get_fs
from segment_speed_utils import helpers, time_series_utils
from segment_speed_utils.project_vars import SCHED_GCS, SEGMENT_GCS, GTFS_DATA_DICT, analysis_date

i'd start with 5, checking if the columns you want are already there in this file (saved out at the end of stop_arrivals_to_speed, and is a speeds by trip table for that day)

In [2]:
catalog = shared_utils.catalog_utils.get_catalog('gtfs_analytics_data')

In [3]:
catalog.speedmap_segments

{'dir': '${gcs_paths.SEGMENT_GCS}', 'stage1': '${speeds_tables.vp_dwell}', 'proxy_stop_times': 'stop_time_expansion/speedmap_stop_times', 'stage2': 'nearest/nearest_vp_speedmap_proxy', 'stage2b': 'nearest/nearest2_vp_speedmap_proxy', 'stage3': 'speedmap/stop_arrivals_proxy', 'stage3b': 'speedmap/stop_arrivals', 'stage4': 'speedmap/speeds', 'trip_stop_cols': ['trip_instance_key', 'stop_sequence', 'stop_sequence1'], 'shape_stop_cols': ['shape_array_key', 'shape_id'], 'stop_pair_cols': ['stop_pair', 'stop_pair_name', 'segment_id'], 'route_dir_cols': ['route_id', 'route_short_name'], 'segments_file': 'segment_options/speedmap_segments', 'shape_stop_single_segment': 'rollup_singleday/speeds_shape_speedmap_segments', 'shape_stop_single_segment_detail': 'rollup_singleday/speeds_shape_speedmap_segments_detail', 'route_dir_single_segment': 'rollup_singleday/speeds_route_dir_speedmap_segments', 'route_dir_multi_segment': 'rollup_multiday/speeds_route_dir_speedmap_segments', 'min_trip_minutes': '

In [4]:
analysis_date

'2024-10-16'

In [5]:
path = f'{catalog.speedmap_segments.dir}{catalog.speedmap_segments.stage4}_{analysis_date}.parquet'

In [6]:
path

'gs://calitp-analytics-data/data-analyses/rt_segment_speeds/speedmap/speeds_2024-10-16.parquet'

In [7]:
df = pd.read_parquet(path)

In [8]:
df.columns

Index(['trip_instance_key', 'stop_sequence', 'stop_sequence1',
       'shape_array_key', 'stop_meters', 'arrival_time', 'arrival_time_sec',
       'subseq_arrival_time_sec', 'subseq_stop_meters', 'meters_elapsed',
       'sec_elapsed', 'speed_mph', 'schedule_gtfs_dataset_key', 'route_id',
       'direction_id', 'route_short_name', 'scheduled_service_minutes',
       'sched_rt_category', 'time_of_day', 'shape_id', 'segment_id',
       'stop_pair_name', 'stop_pair'],
      dtype='object')

In [9]:
df >> head(3)

,trip_instance_key,stop_sequence,stop_sequence1,shape_array_key,stop_meters,arrival_time,arrival_time_sec,subseq_arrival_time_sec,subseq_stop_meters,meters_elapsed,...,route_id,direction_id,route_short_name,scheduled_service_minutes,sched_rt_category,time_of_day,shape_id,segment_id,stop_pair_name,stop_pair
0,00000ec4ec4f4cee317f06c981d4965f,2,2.0,d0a9e8989ac39227513e69a468f002b5,258.571,2024-10-16 12:43:01,45781,45836.0,442.814,184.243,...,3,0.0,3,61.0,schedule_and_vp,Midday,3_0_97,12166-91043-1,Euclid Av & Unity Place Drwy__Euclid Av & Eucl...,12166__91043
1,00000ec4ec4f4cee317f06c981d4965f,3,3.0,d0a9e8989ac39227513e69a468f002b5,442.814,2024-10-16 12:43:56,45836,45894.0,816.841,374.027,...,3,0.0,3,61.0,schedule_and_vp,Midday,3_0_97,91043-12165-1,Euclid Av & Euclid Health Center__Euclid Av & ...,91043__12165
2,00000ec4ec4f4cee317f06c981d4965f,4,4.0,d0a9e8989ac39227513e69a468f002b5,816.841,2024-10-16 12:44:54,45894,45921.0,1248.511,431.670,...,3,0.0,3,61.0,schedule_and_vp,Midday,3_0_97,12165-12164-1,Euclid Av & Brooks Huffman Plaza__Euclid Av & ...,12165__12164


In [10]:
df.time_of_day.unique()

array(['Midday', 'Early AM', 'PM Peak', 'Evening', 'AM Peak', 'Owl'],
      dtype=object)

In [11]:
path = f'{catalog.speedmap_segments.dir}{catalog.speedmap_segments.shape_stop_single_segment_detail}_{analysis_date}.parquet'

In [16]:
path

'gs://calitp-analytics-data/data-analyses/rt_segment_speeds/rollup_singleday/speeds_shape_speedmap_segments_detail_2024-10-16.parquet'

In [14]:
detail = gpd.read_parquet(path)

In [15]:
detail

,schedule_gtfs_dataset_key,shape_array_key,shape_id,route_id,route_short_name,stop_pair,stop_pair_name,segment_id,time_of_day,p50_mph,n_trips,p20_mph,p80_mph,name,caltrans_district,organization_source_record_id,organization_name,base64_url,geometry
0,0139b1253130b33adcd4b3a4490530d2,51530f7b5ba4cdde7c644c14f262005c,b0c096a2-4c32-4675-b005-43b58b3781e6,874b0dfa-ab3d-47da-ac75-5fd7b9bdea42,C30 WB,0d1ef8b5-90e4-40de-b37b-504cb862d18b__5b9a7352...,Woodlake Park EB__Cypress St & Naranjo Blvd WB,0d1ef8b5-90e4-40de-b37b-504cb862d18b-5b9a7352-...,AM Peak,17.27,4,15.20,20.96,TCRTA TripShot Schedule,06 - Fresno,rec4YWVqVwJ9XqDbq,Tulare County Regional Transit Agency,aHR0cHM6Ly90Y3J0YS50cmlwc2hvdC5jb20vdjEvZ3Rmcy...,"LINESTRING (-119.09762 36.41481, -119.09762 36..."
1,0139b1253130b33adcd4b3a4490530d2,51530f7b5ba4cdde7c644c14f262005c,b0c096a2-4c32-4675-b005-43b58b3781e6,874b0dfa-ab3d-47da-ac75-5fd7b9bdea42,C30 WB,0d1ef8b5-90e4-40de-b37b-504cb862d18b__5b9a7352...,Woodlake Park EB__Cypress St & Naranjo Blvd WB,0d1ef8b5-90e4-40de-b37b-504cb862d18b-5b9a7352-...,Midday,20.38,6,18.46,23.02,TCRTA TripShot Schedule,06 - Fresno,rec4YWVqVwJ9XqDbq,Tulare County Regional Transit Agency,aHR0cHM6Ly90Y3J0YS50cmlwc2hvdC5jb20vdjEvZ3Rmcy...,"LINESTRING (-119.09762 36.41481, -119.09762 36..."
2,0139b1253130b33adcd4b3a4490530d2,51530f7b5ba4cdde7c644c14f262005c,b0c096a2-4c32-4675-b005-43b58b3781e6,874b0dfa-ab3d-47da-ac75-5fd7b9bdea42,C30 WB,0d1ef8b5-90e4-40de-b37b-504cb862d18b__5b9a7352...,Woodlake Park EB__Cypress St & Naranjo Blvd WB,0d1ef8b5-90e4-40de-b37b-504cb862d18b-5b9a7352-...,PM Peak,23.90,5,22.49,25.11,TCRTA TripShot Schedule,06 - Fresno,rec4YWVqVwJ9XqDbq,Tulare County Regional Transit Agency,aHR0cHM6Ly90Y3J0YS50cmlwc2hvdC5jb20vdjEvZ3Rmcy...,"LINESTRING (-119.09762 36.41481, -119.09762 36..."
3,0139b1253130b33adcd4b3a4490530d2,51530f7b5ba4cdde7c644c14f262005c,b0c096a2-4c32-4675-b005-43b58b3781e6,874b0dfa-ab3d-47da-ac75-5fd7b9bdea42,C30 WB,5b9a7352-556f-4272-b663-22f9821d6a40__5f894da7...,Cypress St & Naranjo Blvd WB__Millwood and Roa...,5b9a7352-556f-4272-b663-22f9821d6a40-5f894da7-...,AM Peak,49.50,8,43.85,51.81,TCRTA TripShot Schedule,06 - Fresno,rec4YWVqVwJ9XqDbq,Tulare County Regional Transit Agency,aHR0cHM6Ly90Y3J0YS50cmlwc2hvdC5jb20vdjEvZ3Rmcy...,"LINESTRING (-119.10550 36.41377, -119.10562 36..."
4,0139b1253130b33adcd4b3a4490530d2,51530f7b5ba4cdde7c644c14f262005c,b0c096a2-4c32-4675-b005-43b58b3781e6,874b0dfa-ab3d-47da-ac75-5fd7b9bdea42,C30 WB,5b9a7352-556f-4272-b663-22f9821d6a40__5f894da7...,Cypress St & Naranjo Blvd WB__Millwood and Roa...,5b9a7352-556f-4272-b663-22f9821d6a40-5f894da7-...,Midday,47.58,12,44.74,49.68,TCRTA TripShot Schedule,06 - Fresno,rec4YWVqVwJ9XqDbq,Tulare County Regional Transit Agency,aHR0cHM6Ly90Y3J0YS50cmlwc2hvdC5jb20vdjEvZ3Rmcy...,"LINESTRING (-119.10550 36.41377, -119.10562 36..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
693202,fb467982dcc77a7f9199bebe709bb700,8d34a33f017ea0d1498b0cebf26fc5f9,114150,21,21,61591__61592,California & Showers__California & Ortega,61591-61592-1,Evening,24.71,1,24.71,24.71,Bay Area 511 Santa Clara Transit Schedule,04 - Oakland,recC5CT95EufmQCXr,Santa Clara Valley Transportation Authority,aHR0cHM6Ly9hcGkuNTExLm9yZy90cmFuc2l0L2RhdGFmZW...,"LINESTRING (-122.10605 37.40280, -122.10581 37..."
693203,fb467982dcc77a7f9199bebe709bb700,8d34a33f017ea0d1498b0cebf26fc5f9,114150,21,21,61592__61593,California & Ortega__California & Rengstorff,61592-61593-1,Evening,24.68,1,24.68,24.68,Bay Area 511 Santa Clara Transit Schedule,04 - Oakland,recC5CT95EufmQCXr,Santa Clara Valley Transportation Authority,aHR0cHM6Ly9hcGkuNTExLm9yZy90cmFuc2l0L2RhdGFmZW...,"LINESTRING (-122.10339 37.40108, -122.10266 37..."
693204,fb467982dcc77a7f9199bebe709bb700,8d34a33f017ea0d1498b0cebf26fc5f9,114150,21,21,61593__61422,California & Rengstorff__California & Escuela,61593-61422-1,Evening,24.44,1,24.44,24.44,Bay Area 511 Santa Clara Transit Schedule,04 - Oakland,recC5CT95EufmQCXr,Santa Cla